1 - Importação dos pacotes e bibliotecas

In [25]:
# Usada para manipulação de dados
import pandas as pd

# Usado para tranformar strings em dados numéricos
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# Módulos do Scikit-learn usado para algoritimos de regressão
from sklearn.ensemble import GradientBoostingClassifier, RandomForestRegressor
from sklearn.linear_model import LinearRegression

# Módulo do Scikit-learn usado para separar o conjunto de dados.
from sklearn.model_selection import train_test_split

# Módulos do Scikit-learn usado para avaliação estatística.
from sklearn.metrics  import mean_absolute_error
from sklearn.metrics  import mean_squared_error
from sklearn.metrics  import r2_score

2 - Ingenstão dos dados de entrega

In [21]:
# Define o Caminho dos arquivos.
csv_customers = "../data/olist_customers_dataset.csv"
csv_order_itens = "../data/olist_order_items_dataset.csv"
csv_orders = "../data/olist_orders_dataset.csv"
csv_products = "../data/olist_products_dataset.csv"
csv_sellers = "../data/olist_sellers_dataset.csv"

# Carrega os arquivos CSV como DataFrames
df_costumers = pd.read_csv(csv_customers)
df_order_itens = pd.read_csv(csv_order_itens)
df_orders = pd.read_csv(csv_orders)
df_products = pd.read_csv(csv_products)
df_sellers = pd.read_csv(csv_sellers)

# Junta os dataframes em um unico dataframe para utilizar no machine learning.
df = df_orders.merge(df_order_itens, on="order_id")
df = df.merge(df_costumers, on="customer_id")
df = df.merge(df_sellers, on='seller_id')
df = df.merge(df_products, on="product_id")


# Exibe o dataframe
df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,...,seller_city,seller_state,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1,87285b34884572647811a353c7ac498a,...,maua,SP,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,1,595fac2a385ac33a80bd5114aec74eb8,...,belo horizonte,SP,perfumaria,29.0,178.0,1.0,400.0,19.0,13.0,19.0
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,1,aa4383b373c6aca5d8797843e5594415,...,guariba,SP,automotivo,46.0,232.0,1.0,420.0,24.0,19.0,21.0
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,1,d0b61bfb1de832b15ba9d266ca96e5b0,...,belo horizonte,MG,pet_shop,59.0,468.0,3.0,450.0,30.0,10.0,20.0
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,1,65266b2da20d04dbe00c5c2d3bb7859e,...,mogi das cruzes,SP,papelaria,38.0,316.0,4.0,250.0,51.0,15.0,15.0


3 - Tratamento dos dados

In [22]:
# Tira todas as colunas irrelevantes para o aprendizado de maquina.
df = df.drop(['order_id', 'customer_id','order_status', 'product_id', 'order_item_id', 'seller_id', 'customer_unique_id', 'product_name_lenght', 'product_description_lenght', "product_photos_qty", "order_approved_at", 'order_delivered_carrier_date', 'order_estimated_delivery_date', 'shipping_limit_date'], axis=1)


In [23]:
# Remove as linhas que contenham algum valor nulo.
df = df.dropna()

# Trasforma as colunas com strings em valores numeros para o aprendizado de maquina.
colunas_categoricas = ['customer_state', 'seller_state', 'customer_city', 'seller_city', 'product_category_name']
for col in colunas_categoricas:
    df[col] = le.fit_transform(df[col].astype(str))


In [24]:
# Converte as datas para datetime
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
df['order_delivered_customer_date'] = pd.to_datetime(df['order_delivered_customer_date'])

# Cria a Variável alvo: diferença em dias entre a entrega e a compra.
df['prazo_entrega_dias'] = (df['order_delivered_customer_date'] - df['order_purchase_timestamp'])

# Converte para um dado numérico.
df['prazo_entrega_dias'] = df['prazo_entrega_dias'].dt.days

# Exclui as colunas de data de compra e data de entrega.
df = df.drop(['order_purchase_timestamp', 'order_delivered_customer_date'] , axis=1)

In [19]:
# Exibe o dataframe
display(df)

# Exibe informações detalhadas sobre a estrutura dos dados.
display(df.info())

# Exibe a quantidade de valores nulos por coluna.
print(df.isnull().sum())


,price,freight_value,customer_zip_code_prefix,customer_city,customer_state,seller_zip_code_prefix,seller_city,seller_state,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm,prazo_entrega_dias
0,29.99,8.72,3149,3554,25,9350,318,21,72,500.0,19.0,8.0,13.0,8 days 10:28:40
1,118.70,22.76,47813,413,4,31570,61,21,62,400.0,19.0,13.0,19.0,13 days 18:46:08
2,159.90,19.22,75265,4005,8,14840,212,21,8,420.0,24.0,19.0,21.0,9 days 09:27:40
3,45.00,27.20,59296,3432,19,31842,61,7,63,450.0,30.0,10.0,20.0,13 days 05:00:36
4,19.90,8.72,9195,3338,25,8752,328,21,59,250.0,51.0,15.0,15.0,2 days 20:58:23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112645,174.90,20.10,11722,2966,25,17602,556,21,9,4950.0,40.0,10.0,40.0,22 days 04:38:58
112646,205.99,65.02,45920,2516,4,8290,498,21,29,13300.0,32.0,90.0,22.0,24 days 20:37:34
112647,179.99,40.59,28685,1913,18,37175,225,7,44,6550.0,20.0,20.0,20.0,17 days 02:04:27
112648,179.99,40.59,28685,1913,18,37175,225,7,44,6550.0,20.0,20.0,20.0,17 days 02:04:27


<class 'pandas.core.frame.DataFrame'>
Index: 108658 entries, 0 to 112649
Data columns (total 14 columns):
 #   Column                    Non-Null Count   Dtype          
---  ------                    --------------   -----          
 0   price                     108658 non-null  float64        
 1   freight_value             108658 non-null  float64        
 2   customer_zip_code_prefix  108658 non-null  int64          
 3   customer_city             108658 non-null  int64          
 4   customer_state            108658 non-null  int64          
 5   seller_zip_code_prefix    108658 non-null  int64          
 6   seller_city               108658 non-null  int64          
 7   seller_state              108658 non-null  int64          
 8   product_category_name     108658 non-null  int64          
 9   product_weight_g          108658 non-null  float64        
 10  product_length_cm         108658 non-null  float64        
 11  product_height_cm         108658 non-null  float64       

None

price                       0
freight_value               0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
seller_zip_code_prefix      0
seller_city                 0
seller_state                0
product_category_name       0
product_weight_g            0
product_length_cm           0
product_height_cm           0
product_width_cm            0
prazo_entrega_dias          0
dtype: int64


4 - Separação dos dados